# Java API / RapidAPI
> API Data
- toc: false
- comments: true
- categories: [jupyter]

In [4]:
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;

//RapidAPI header  https://rapidapi.com/spamakashrajtech/api/corona-virus-world-and-india-data
HttpRequest request = HttpRequest.newBuilder()
		.uri(URI.create("https://earthquake-monitor.p.rapidapi.com/recent"))
		.header("X-RapidAPI-Key", "cb6c4ae0c0mshf7c680cd7f9687bp1c11edjsnd948864f5be3")
		.header("X-RapidAPI-Host", "earthquake-monitor.p.rapidapi.com")
		.method("GET", HttpRequest.BodyPublishers.noBody())
		.build();
HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());
System.out.println(response.body());

[{"event_number":"1175154","link":"https://www.emsc-csem.org/Earthquake/earthquake.php?id=1175154","date":"2022-10-03","time":"07:07:52.8","time_ago":"20min ago","latitude":"32.96 N","longitude":"115.88 W","depth":"11 km","magnitude":"2.3","magnitude_type":"Ml","location":"SOUTHERN CALIFORNIA","testimonies":"","testimonies_link":"","updated":"2022-10-03 07:11"},{"event_number":"1175155","link":"https://www.emsc-csem.org/Earthquake/earthquake.php?id=1175155","date":"2022-10-03","time":"07:00:43.5","time_ago":"27min ago","latitude":"56.44 S","longitude":"26.04 W","depth":"40 km","magnitude":"5.5","magnitude_type":"Mw","location":"SOUTH SANDWICH ISLANDS REGION","testimonies":"","testimonies_link":"","updated":"2022-10-03 07:21"},{"event_number":"1175149","link":"https://www.emsc-csem.org/Earthquake/earthquake.php?id=1175149","date":"2022-10-03","time":"06:52:17.0","time_ago":"35min ago","latitude":"10.85 N","longitude":"85.13 W","depth":"6 km","magnitude":"2.9","magnitude_type":"M","locat

In [ ]:
package com.nighthawk.spring_portfolio.mvc.covid;

import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.util.Date;
import java.util.HashMap;

import org.json.simple.JSONObject;
import org.json.simple.parser.JSONParser;
import org.springframework.http.HttpStatus;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

@RestController // annotation to create a RESTful web services
@RequestMapping("/api/covid")  //prefix of API
public class CovidApiController {
    private JSONObject body; //last run result
    private HttpStatus status; //last run status
    String last_run = null; //last run day of month

    // GET Covid 19 Stats
    @GetMapping("/daily")   //added to end of prefix as endpoint
    public ResponseEntity<JSONObject> getCovid() {

        //calls API once a day, sets body and status properties
        String today = new Date().toString().substring(0,10); 
        if (last_run == null || !today.equals(last_run))
        {
            try {  //APIs can fail (ie Internet or Service down)

                //RapidAPI header
                HttpRequest request = HttpRequest.newBuilder()
                    .uri(URI.create("https://corona-virus-world-and-india-data.p.rapidapi.com/api"))
                    .header("x-rapidapi-key", "dec069b877msh0d9d0827664078cp1a18fajsn2afac35ae063")
                    .header("x-rapidapi-host", "corona-virus-world-and-india-data.p.rapidapi.com")
                    .method("GET", HttpRequest.BodyPublishers.noBody())
                    .build();

                //RapidAPI request and response
                HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());

                //JSONParser extracts text body and parses to JSONObject
                this.body = (JSONObject) new JSONParser().parse(response.body());
                this.status = HttpStatus.OK;  //200 success
                this.last_run = today;
            }
            catch (Exception e) {  //capture failure info
                HashMap<String, String> status = new HashMap<>();
                status.put("status", "RapidApi failure: " + e);

                //Setup object for error
                this.body = (JSONObject) status;
                this.status = HttpStatus.INTERNAL_SERVER_ERROR; //500 error
                this.last_run = null;
            }
        }

        //return JSONObject in RESTful style
        return new ResponseEntity<>(body, status);
    }
}